## 🛳️ Titanic - Random Forest
Os modelos Random Forest Classifier são utilizados para capturar relações não lineares e interações complexas entre as variáveis. Por serem modelos baseados em árvores não assumem linearidade e são robustos a outliers;

### Destaques do Notebook  

* **Pré-processamento inicial:**.

* **Transformação das variáveis:** a

* **Treinamento do modelo:** construção de um pipeline com o RandomForest,sem ajustes hiperparâmetros e com ajuste. 

## 1. Bibliotecas

In [26]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import joblib
from pathlib import Path

# sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin, clone

# Importações locais
from setup_notebook import setup_path
setup_path()
from src.model_utils import *
from src.preprocess_utils_tic import preprocessador_titanic


## 2. Dataload & Preprocessamento com joblib

In [32]:
BASE = Path.cwd().parent   
# =====================================================
# ⚙️ 0. carregamento dos preprocessador 
# =====================================================
temp = joblib.load(BASE /'src'/'preprocess_Titanic_v1.0.joblib')
PP2=temp['preprocessador']

# # =====================================================
# # 📁 1. Leitura dos dados & Separação das bases
# # =====================================================

DATA_DIR = BASE / "data" / "processed"
X_train = pd.read_csv(DATA_DIR / "X_train_final.csv").reset_index(drop=True)
X_test  = pd.read_csv(DATA_DIR / "X_test_final.csv")
y_train = pd.read_csv(DATA_DIR / "y_train_final.csv").values.ravel()
y_test  = pd.read_csv(DATA_DIR / "y_test_final.csv")


# # =====================================================
# #  🤖 3.Definição dos Modelos
# # =====================================================
model_RF0 = RandomForestClassifier(random_state=42, n_jobs=-1)  #Baseline
pipe_RF0      = pipe_models(model_RF0,PP2)

model_RF1 = RandomForestClassifier(max_depth=8, max_features=None,min_samples_leaf=2,
                                   min_samples_split=11, n_estimators=144,random_state=42)
pipe_RF1      = pipe_models(model_RF1,PP2)

model_RF2 = RandomForestClassifier(ccp_alpha=0.0002878697725951174, criterion='entropy',
                       max_depth=10, max_features=None, min_samples_leaf=2,
                       min_samples_split=9, n_estimators=213,random_state=42)
pipe_RF2      = pipe_models(model_RF2,PP2)




pipe_RF3 = joblib.load('modelo_RF_final_bayes.joblib')
pipe_RF3.named_steps # informações sobre a pipeline

# Situação com leitura dos parametros apenas 
# best_params  = joblib.load('parametros_RF_BAYER.joblib')
# rf_params = {k.replace('model__', ''): v for k, v in best_params.items()}
# model_RF3 = RandomForestClassifier(**rf_params, random_state=42, n_jobs=-1)
# pipe_RF3      = pipe_models(model_RF3,PP2)

## 3.Treinamento

In [28]:
# Baseline
baseline_scores = cross_val_score(pipe_RF0 , X_train, y_train, cv=10)

# 2. Testa a performance 
pipe_RF0.fit(X_train, y_train)
y_pred=pipe_RF0.predict(X_test)
print(f"{'='*70}")
print(f"🎯 Random Forest (Baseline) | cvscores : {baseline_scores.mean():.4f} ± {baseline_scores.std():.4f}")
print(f"{'='*70}")
print(f"📊 **Acurácia no Teste**: {accuracy_score(y_test, y_pred):.4f}")
print(f"\n📋 **Relatório de Classificação**:")
print(classification_report(y_test, y_pred))
cm=confusion_matrix(y_test, y_pred)
print(f"🎯 **Matriz de Confusão**:")
print(f"               Previsto 0   Previsto 1")
print(f"Real 0         {cm[0,0]:<11} {cm[0,1]:<11}")
print(f"Real 1         {cm[1,0]:<11} {cm[1,1]:<11}")
print(f"{'─'*70}")

🎯 Random Forest (Baseline) | cvscores : 0.8139 ± 0.0475
📊 **Acurácia no Teste**: 0.7836

📋 **Relatório de Classificação**:
              precision    recall  f1-score   support

           0       0.80      0.83      0.82       157
           1       0.75      0.71      0.73       111

    accuracy                           0.78       268
   macro avg       0.78      0.77      0.78       268
weighted avg       0.78      0.78      0.78       268

🎯 **Matriz de Confusão**:
               Previsto 0   Previsto 1
Real 0         131         26         
Real 1         32          79         
──────────────────────────────────────────────────────────────────────


In [29]:
# RF1
cv_scores = cross_val_score(pipe_RF1 , X_train, y_train, cv=10)

# 2. Testa a performance 
pipe_RF1.fit(X_train, y_train)
y_pred=pipe_RF1.predict(X_test)
print(f"{'='*70}")
print(f"🎯 Random Forest 1 | cvscores : {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
print(f"{'='*70}")
print(f"📊 **Acurácia no Teste**: {accuracy_score(y_test, y_pred):.4f}")
print(f"\n📋 **Relatório de Classificação**:")
print(classification_report(y_test, y_pred))
cm=confusion_matrix(y_test, y_pred)
print(f"🎯 **Matriz de Confusão**:")
print(f"               Previsto 0   Previsto 1")
print(f"Real 0         {cm[0,0]:<11} {cm[0,1]:<11}")
print(f"Real 1         {cm[1,0]:<11} {cm[1,1]:<11}")
print(f"{'─'*70}")

🎯 Random Forest 1 | cvscores : 0.8348 ± 0.0413
📊 **Acurácia no Teste**: 0.7948

📋 **Relatório de Classificação**:
              precision    recall  f1-score   support

           0       0.79      0.89      0.83       157
           1       0.80      0.67      0.73       111

    accuracy                           0.79       268
   macro avg       0.80      0.78      0.78       268
weighted avg       0.80      0.79      0.79       268

🎯 **Matriz de Confusão**:
               Previsto 0   Previsto 1
Real 0         139         18         
Real 1         37          74         
──────────────────────────────────────────────────────────────────────


In [30]:
#RF 2
cv_scores = cross_val_score(pipe_RF2 , X_train, y_train, cv=10)

# 2. Testa a performance 
pipe_RF2.fit(X_train, y_train)
y_pred=pipe_RF2.predict(X_test)
print(f"{'='*70}")
print(f"🎯 Random Forest 2 | cvscores : {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
print(f"{'='*70}")
print(f"📊 **Acurácia no Teste**: {accuracy_score(y_test, y_pred):.4f}")
print(f"\n📋 **Relatório de Classificação**:")
print(classification_report(y_test, y_pred))
cm=confusion_matrix(y_test, y_pred)
print(f"🎯 **Matriz de Confusão**:")
print(f"               Previsto 0   Previsto 1")
print(f"Real 0         {cm[0,0]:<11} {cm[0,1]:<11}")
print(f"Real 1         {cm[1,0]:<11} {cm[1,1]:<11}")
print(f"{'─'*70}")

🎯 Random Forest 2 | cvscores : 0.8299 ± 0.0399
📊 **Acurácia no Teste**: 0.7836

📋 **Relatório de Classificação**:
              precision    recall  f1-score   support

           0       0.79      0.86      0.82       157
           1       0.77      0.68      0.72       111

    accuracy                           0.78       268
   macro avg       0.78      0.77      0.77       268
weighted avg       0.78      0.78      0.78       268

🎯 **Matriz de Confusão**:
               Previsto 0   Previsto 1
Real 0         135         22         
Real 1         36          75         
──────────────────────────────────────────────────────────────────────


In [31]:
#RF 3
cv_scores = cross_val_score(pipe_RF3 , X_train, y_train, cv=10)

# 2. Testa a performance 
pipe_RF3.fit(X_train, y_train)
y_pred=pipe_RF3.predict(X_test)
print(f"{'='*70}")
print(f"🎯 Random Forest 2 | cvscores : {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
print(f"{'='*70}")
print(f"📊 **Acurácia no Teste**: {accuracy_score(y_test, y_pred):.4f}")
print(f"\n📋 **Relatório de Classificação**:")
print(classification_report(y_test, y_pred))
cm=confusion_matrix(y_test, y_pred)
print(f"🎯 **Matriz de Confusão**:")
print(f"               Previsto 0   Previsto 1")
print(f"Real 0         {cm[0,0]:<11} {cm[0,1]:<11}")
print(f"Real 1         {cm[1,0]:<11} {cm[1,1]:<11}")
print(f"{'─'*70}")

🎯 Random Forest 2 | cvscores : 0.8444 ± 0.0610
📊 **Acurácia no Teste**: 0.8060

📋 **Relatório de Classificação**:
              precision    recall  f1-score   support

           0       0.81      0.88      0.84       157
           1       0.80      0.70      0.75       111

    accuracy                           0.81       268
   macro avg       0.81      0.79      0.80       268
weighted avg       0.81      0.81      0.80       268

🎯 **Matriz de Confusão**:
               Previsto 0   Previsto 1
Real 0         138         19         
Real 1         33          78         
──────────────────────────────────────────────────────────────────────
